<a href="https://colab.research.google.com/github/ishanmalviya10/NASA/blob/Ishan-(QML-algorithms)/CompleteNASAcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# QML for Air Quality - Google Colab
# ============================================

# Install packages
!pip install qiskit qiskit-machine-learning qiskit-aer qiskit-optimization -q
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost -q

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, silhouette_score, davies_bouldin_score
from sklearn.svm import SVC, SVR
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from xgboost import XGBRegressor, XGBClassifier
from scipy.linalg import eigh
import warnings
warnings.filterwarnings('ignore')

# Qiskit imports
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import VQC, QSVC, NeuralNetworkRegressor
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit import transpile

print("✅ Setup complete!")
print(f"Qiskit version: {qiskit.__version__}")

# ============================================
# LOAD CSV DATA
# ============================================

from google.colab import files
print("📂 Upload your CSV file:")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print(f"\n✅ Loaded dataset: {df.shape}")
print(df.head())
print(f"\nColumns: {df.columns.tolist()}")

# Configure target column
print("\n⚙️ Configure target column:")
print(f"Available columns: {df.columns.tolist()}")
target_col = input("Enter target column name (or press Enter for last column): ").strip()
if not target_col:
    target_col = df.columns[-1]
print(f"Target column: {target_col}")

# Prepare features and target
feature_cols = [col for col in df.columns if col != target_col]
X = df[feature_cols].values
y_reg = df[target_col].values

# Binary classification target (median threshold)
threshold = np.median(y_reg)
y_class = (y_reg > threshold).astype(int)
print(f"\nClassification threshold: {threshold:.2f}")
print(f"Class distribution: {np.bincount(y_class)}")

# Train/test split
X_train, X_test, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)
_, _, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)

# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n📦 Training set: {X_train.shape}")
print(f"📦 Test set: {X_test.shape}")

# ============================================
# CLASSICAL BASELINE: XGBoost
# ============================================

print("\n🔄 Training XGBoost Regressor...")
xgb_reg = XGBRegressor(n_estimators=300, max_depth=6, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, random_state=42)
xgb_reg.fit(X_train_scaled, y_train_reg)
y_pred_reg = xgb_reg.predict(X_test_scaled)

mae = mean_absolute_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"\n📈 XGBoost Regression:")
print(f"  MAE:  {mae:.2f}")
print(f"  RMSE: {rmse:.2f}")
print(f"  R²:   {r2:.3f}")

print("\n🔄 Training XGBoost Classifier...")
xgb_class = XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.05, random_state=42)
xgb_class.fit(X_train_scaled, y_train_class)
y_pred_class = xgb_class.predict(X_test_scaled)

print("\n📊 XGBoost Classification:")
print(classification_report(y_test_class, y_pred_class, target_names=['Class 0', 'Class 1']))

# Feature importance
importance = xgb_reg.feature_importances_
feature_importance = pd.DataFrame({'feature': feature_cols, 'importance': importance}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# ============================================
# QUANTUM SVM (QSVM)
# ============================================

# Select top 4 features for quantum
top_features_idx = feature_importance.head(4).index.tolist()
top_features = [feature_cols[i] for i in range(len(feature_cols)) if i in top_features_idx]

X_qml = X[:, top_features_idx]
X_train_qml, X_test_qml, y_train_qml, y_test_qml = train_test_split(X_qml, y_class, test_size=0.2, random_state=42)

# Normalize
scaler_qml = StandardScaler()
X_train_qml_scaled = scaler_qml.fit_transform(X_train_qml)
X_test_qml_scaled = scaler_qml.transform(X_test_qml)

# Map to [0, π]
X_train_qml_norm = (X_train_qml_scaled - X_train_qml_scaled.min()) / (X_train_qml_scaled.max() - X_train_qml_scaled.min()) * np.pi
X_test_qml_norm = (X_test_qml_scaled - X_test_qml_scaled.min()) / (X_test_qml_scaled.max() - X_test_qml_scaled.min()) * np.pi

# Subset for quantum (computational limit)
n_train_q = min(100, len(X_train_qml_norm))
n_test_q = min(40, len(X_test_qml_norm))

X_train_q = X_train_qml_norm[:n_train_q]
y_train_q = y_train_qml[:n_train_q]
X_test_q = X_test_qml_norm[:n_test_q]
y_test_q = y_test_qml[:n_test_q]

print(f"\n📦 Quantum training: {X_train_q.shape}")
print(f"📦 Quantum test: {X_test_q.shape}")

# Build quantum kernel
num_qubits = X_train_q.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2, entanglement='linear')

print("\n🎨 Quantum Feature Map:")
feature_map.decompose().draw('mpl')
plt.title(f'ZZ Feature Map ({num_qubits} qubits)')
plt.show()

quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

print("\n🔄 Training Quantum SVM (2-5 mins)...")
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(X_train_q, y_train_q)

y_pred_q = qsvc.predict(X_test_q)

qsvm_accuracy = accuracy_score(y_test_q, y_pred_q)
qsvm_precision = precision_score(y_test_q, y_pred_q)
qsvm_recall = recall_score(y_test_q, y_pred_q)
qsvm_f1 = f1_score(y_test_q, y_pred_q)

print(f"\n🎯 Quantum SVM:")
print(f"  Accuracy:  {qsvm_accuracy:.3f}")
print(f"  Precision: {qsvm_precision:.3f}")
print(f"  Recall:    {qsvm_recall:.3f}")
print(f"  F1-Score:  {qsvm_f1:.3f}")

# Classical SVM comparison
classical_svc = SVC(kernel='rbf', C=1.0, gamma='scale')
classical_svc.fit(X_train_q, y_train_q)
y_pred_classical = classical_svc.predict(X_test_q)

classical_accuracy = accuracy_score(y_test_q, y_pred_classical)
classical_f1 = f1_score(y_test_q, y_pred_classical)

print(f"\n⚖️ Classical vs Quantum:")
print(f"  Classical SVM Accuracy: {classical_accuracy:.3f}")
print(f"  Quantum SVM Accuracy:   {qsvm_accuracy:.3f}")

# ============================================
# VARIATIONAL QUANTUM CLASSIFIER (VQC)
# ============================================

print("\n🔄 Building VQC...")

feature_map_vqc = ZZFeatureMap(feature_dimension=num_qubits, reps=1)
ansatz = RealAmplitudes(num_qubits=num_qubits, reps=3)

print(f"\n⚙️ Variational parameters: {ansatz.num_parameters}")

print("\n🔄 Training VQC (5-10 mins)...")
vqc = VQC(
    feature_map=feature_map_vqc,
    ansatz=ansatz,
    optimizer=COBYLA(maxiter=100),
    callback=lambda weights, obj_val: print(f"  Objective: {obj_val:.4f}")
)

vqc.fit(X_train_q, y_train_q)
y_pred_vqc = vqc.predict(X_test_q)

vqc_accuracy = accuracy_score(y_test_q, y_pred_vqc)
vqc_f1 = f1_score(y_test_q, y_pred_vqc, zero_division=0)

print(f"\n🎯 VQC:")
print(f"  Accuracy: {vqc_accuracy:.3f}")
print(f"  F1-Score: {vqc_f1:.3f}")

# ============================================
# QUANTUM PCA
# ============================================

print("\n🔄 Quantum-Inspired PCA...")

X_pca = X_train_scaled[:200]

classical_pca = PCA(n_components=4)
X_pca_classical = classical_pca.fit_transform(X_pca)

print(f"\n📊 Classical PCA:")
print(f"  Explained variance: {classical_pca.explained_variance_ratio_}")

def quantum_inspired_pca(X, n_components=4):
    cov_matrix = np.cov(X, rowvar=False)
    eigenvalues, eigenvectors = eigh(cov_matrix)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    principal_components = eigenvectors[:, :n_components]
    X_transformed = X @ principal_components
    explained_variance_ratio = eigenvalues[:n_components] / eigenvalues.sum()
    return X_transformed, explained_variance_ratio, principal_components

X_qpca, qpca_var_ratio, qpca_components = quantum_inspired_pca(X_pca, n_components=4)

print(f"\n🔮 Q-Inspired PCA:")
print(f"  Explained variance: {qpca_var_ratio}")

# ============================================
# QUANTUM K-MEANS
# ============================================

print("\n🔄 Quantum-Inspired K-Means...")

X_cluster = X_pca_classical
n_clusters = 3

classical_kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
labels_classical = classical_kmeans.fit_predict(X_cluster)

def quantum_inspired_kmeans(X, n_clusters=3, max_iter=100):
    np.random.seed(42)
    n_samples, n_features = X.shape
    idx = np.random.choice(n_samples, n_clusters, replace=False)
    centroids = X[idx]

    for iteration in range(max_iter):
        distances = np.zeros((n_samples, n_clusters))
        for i in range(n_clusters):
            X_norm = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-10)
            c_norm = centroids[i] / (np.linalg.norm(centroids[i]) + 1e-10)
            fidelity = np.abs(X_norm @ c_norm) ** 2
            distances[:, i] = 1 - fidelity

        labels = np.argmin(distances, axis=1)
        new_centroids = np.array([X[labels == k].mean(axis=0) for k in range(n_clusters)])

        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids

    return labels, centroids

labels_quantum, centroids_quantum = quantum_inspired_kmeans(X_cluster, n_clusters=3)

sil_classical = silhouette_score(X_cluster, labels_classical)
sil_quantum = silhouette_score(X_cluster, labels_quantum)

print(f"\n📊 Clustering:")
print(f"  Classical K-Means Silhouette: {sil_classical:.3f}")
print(f"  Q-Inspired K-Means Silhouette: {sil_quantum:.3f}")

# ============================================
# QUANTUM FEATURE SELECTION
# ============================================

print("\n🔄 Quantum-Inspired Feature Selection...")

from sklearn.feature_selection import mutual_info_regression

mi_scores = mutual_info_regression(X_train_scaled, y_train_reg, random_state=42)

def quantum_inspired_feature_selection(X, y, n_select=5):
    n_features = X.shape[1]
    corr_matrix = np.corrcoef(X.T)
    selected = []
    remaining = list(range(n_features))

    for _ in range(n_select):
        best_score = -np.inf
        best_feature = None

        for feat in remaining:
            info = mi_scores[feat]
            if selected:
                redundancy = np.mean([abs(corr_matrix[feat, s]) for s in selected])
            else:
                redundancy = 0
            score = info - 0.5 * redundancy

            if score > best_score:
                best_score = score
                best_feature = feat

        selected.append(best_feature)
        remaining.remove(best_feature)

    return selected

selected_features_idx = quantum_inspired_feature_selection(X_train_scaled, y_train_reg, n_select=min(5, len(feature_cols)))
selected_features = [feature_cols[i] for i in selected_features_idx]

print(f"\n🎯 Selected Features: {selected_features}")

# ============================================
# QUANTUM NEURAL NETWORK (QNN)
# ============================================

print("\n🔄 Building QNN...")

n_qubits_qnn = min(4, len(selected_features_idx))
X_train_qnn = X_train_scaled[:100, selected_features_idx[:n_qubits_qnn]]
X_test_qnn = X_test_scaled[:40, selected_features_idx[:n_qubits_qnn]]
y_train_qnn = y_train_reg[:100]
y_test_qnn = y_test_reg[:40]

y_train_qnn_norm = (y_train_qnn - y_train_qnn.min()) / (y_train_qnn.max() - y_train_qnn.min()) * 2 - 1
y_test_qnn_norm = (y_test_qnn - y_train_qnn.min()) / (y_train_qnn.max() - y_train_qnn.min()) * 2 - 1

X_train_qnn_norm = (X_train_qnn - X_train_qnn.min(axis=0)) / (X_train_qnn.max(axis=0) - X_train_qnn.min(axis=0) + 1e-10) * np.pi
X_test_qnn_norm = (X_test_qnn - X_train_qnn.min(axis=0)) / (X_train_qnn.max(axis=0) - X_train_qnn.min(axis=0) + 1e-10) * np.pi

def create_qnn_circuit(n_qubits):
    input_params = [Parameter(f'x{i}') for i in range(n_qubits)]
    weight_params = [Parameter(f'θ{i}') for i in range(n_qubits * 3)]
    qc = QuantumCircuit(n_qubits)

    for i in range(n_qubits):
        qc.ry(input_params[i], i)

    for layer in range(3):
        for i in range(n_qubits):
            qc.ry(weight_params[layer * n_qubits + i], i)
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)
        if n_qubits > 2:
            qc.cx(n_qubits - 1, 0)

    return qc, input_params, weight_params

qnn_circuit, input_params, weight_params = create_qnn_circuit(n_qubits_qnn)

observable = SparsePauliOp.from_list([("Z" * n_qubits_qnn, 1)])

qnn = EstimatorQNN(
    circuit=qnn_circuit,
    observables=[observable],
    input_params=input_params,
    weight_params=weight_params
)

print("\n🔄 Training QNN (5-10 mins)...")
initial_weights = np.random.rand(len(weight_params)) * 0.1

qnn_regressor = NeuralNetworkRegressor(
    neural_network=qnn,
    optimizer=COBYLA(maxiter=50),
    initial_point=initial_weights,
    callback=lambda weights, obj_val: print(f"  Objective: {obj_val:.4f}")
)

qnn_regressor.fit(X_train_qnn_norm, y_train_qnn_norm)
y_pred_qnn_norm = qnn_regressor.predict(X_test_qnn_norm)

y_pred_qnn = (y_pred_qnn_norm + 1) / 2 * (y_train_qnn.max() - y_train_qnn.min()) + y_train_qnn.min()

mae_qnn = mean_absolute_error(y_test_qnn, y_pred_qnn)
r2_qnn = r2_score(y_test_qnn, y_pred_qnn)

print(f"\n🎯 QNN:")
print(f"  MAE: {mae_qnn:.2f}")
print(f"  R²:  {r2_qnn:.3f}")

# ============================================
# RESULTS SUMMARY
# ============================================

print("\n" + "="*60)
print("📊 FINAL RESULTS SUMMARY")
print("="*60)

results = pd.DataFrame({
    'Model': ['Classical XGBoost', 'Classical SVM', 'Quantum SVM', 'VQC', 'QNN'],
    'Accuracy/R²': [r2, classical_accuracy, qsvm_accuracy, vqc_accuracy, r2_qnn],
    'MAE/F1': [mae, classical_f1, qsvm_f1, vqc_f1, mae_qnn]
})

print("\n" + results.to_string(index=False))

# Export results
results.to_csv('qml_results.csv', index=False)
print("\n✅ Saved: qml_results.csv")

print("\n🎉 QML Pipeline Complete!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 20.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 17.2 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'qiskit_algorithms'